In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
#
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import os
print(os.getcwd())

/Users/pavankakarrot/Documents/data_analysis/Cohort_Analysis


In [11]:
df = pd.read_excel('xxxxxxxxx/Cohort_Analysis/Online Retail.xlsx')

In [12]:
df.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
204406,554658,84006,MAGIC TREE -PAPER FLOWERS,12,2011-05-25 14:18:00,0.85,12839.0,United Kingdom
474579,576840,90013C,GREEN VINTAGE EARRINGS,2,2011-11-16 15:23:00,4.15,NaN,United Kingdom
231859,557286,79051A,SMOKEY GREY COLOUR D.O.F. GLASS,4,2011-06-19 12:36:00,0.29,17879.0,United Kingdom
354822,567901,23425,STORAGE TIN HOME SWEET HOME,5,2011-09-22 16:28:00,4.96,NaN,United Kingdom
67071,541827,21015,DARK BIRD HOUSE TREE DECORATION,2,2011-01-21 17:05:00,1.25,NaN,United Kingdom


In [13]:
display(df.describe())

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [14]:
df.drop_duplicates(inplace=True)
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0) & (df['CustomerID'].notnull())]

In [15]:
df.shape

(392692, 8)

In [16]:
def prepare_cohort_data(df):
    """
    Prepare dataframe for cohort analysis by creating necessary date and customer fields
    
    Parameters:
    df (pandas.DataFrame): Cleaned dataframe with CustomerID and InvoiceDate
    
    Returns:
    pandas.DataFrame: Processed dataframe ready for cohort analysis
    """
    # Create TotalAmount column
    df['TotalAmount'] = df['Quantity'] * df['UnitPrice']
    
    # Convert InvoiceDate to datetime if it isn't already
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    
    # Create date fields
    df['InvoiceYearMonth'] = df['InvoiceDate'].dt.to_period('M')
    
    # Get the first purchase date for each customer
    customer_first_purchase = df.groupby('CustomerID')['InvoiceDate'].min().reset_index()
    customer_first_purchase.columns = ['CustomerID', 'FirstPurchaseDate']
    
    # Add first purchase date to main dataframe
    df = df.merge(customer_first_purchase, on='CustomerID')
    
    # Create CohortYearMonth (customer's first purchase month)
    df['CohortYearMonth'] = df['FirstPurchaseDate'].dt.to_period('M')
    
    # Calculate number of months since first purchase
    df['MonthsFromFirstPurchase'] = (
        (df['InvoiceYearMonth'].astype(str).map(lambda x: dt.datetime.strptime(str(x), "%Y-%m"))
         - df['CohortYearMonth'].astype(str).map(lambda x: dt.datetime.strptime(str(x), "%Y-%m")))
        .map(lambda x: x.days/30).astype(int)
    )
    
    return df

# Use the function on your cleaned dataframe
cohort_df = prepare_cohort_data(df)

# Save the processed file
cohort_df.to_csv('retail_cohort_analysis.csv', index=False)